Power Method
-------------

In [1]:
import numpy as np
import xarray as xr


ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = bio_1.values

True


Functions that compute:
* the approximative, normalized eigenvector corresponding to the eigenvalue of largest magnitude (function from Wikipedia)
* the eigenvalue corresponding to given, normalized eigenvector (my own code)
* diagnostics to see how well the power method performs (my own code)

In [2]:
#!/usr/bin/env python3

import numpy as np

def power_iteration(A, iterations: int):
    # A slightly modified version of
    # the function retrieved from 
    # https://en.wikipedia.org/wiki/Power_iteration
    # (Retrieved: 8.9.2022)
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])
    b_k_norm = np.linalg.norm(b_k)
    #normalize
    b_k = b_k/b_k_norm

    for _ in np.arange(iterations):
        # calculate the matrix-by-vector product Ab
        b_k1 = A.dot(b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

def rayleigh_quotient(A, v):
    """Function computes the rayleigh quotient for 2-d numpy array A and normalized vector v.
    (Because v normalized, the denominator v.T.dot(v)=1, and can thus be omitted.)
    If v is an eigenvector of A, the quotient gives the corresponding eigenvalue."""
    mu = v.T.dot(A).dot(v)
    return mu

def residual_norm(A, v, mu):
    """Compute the 2-norm of the residual vector r = Av-av, where
    A is a 2-d numpy array, v an approximated eigenvector and a is an approximation of
    corresponding eigenvalue. Can be used to evaluate the accuracy of the estimated
    eigenvector and -value."""
    r = A.dot(v)-mu*v
    norm = np.linalg.norm(r)
    return norm

Hotelling's deflation
-----------------------
A method for finding the next eigenvectors after the first one 

In [3]:

def hotelling2(A, v, mu, n_iterations):
    """Function to compute the second eigenvector after the largest
    eigenvalue and corresponding eigenvector has been computed.
    Function utilizes Hotelling's method.
    Parameters:
    A : 2D array
    v : 1D array. Eigenvector corresponding to the largest eigenvalue of A. 
    a : float. Largest eigenvalue of A.
    num_simulations : int"""
    b_k = np.random.rand(A.shape[1])
    b_k_norm = np.linalg.norm(b_k)
    #normalize
    b_k = b_k/b_k_norm

    for i in np.arange(n_iterations):
        b_k1 = A.dot(b_k)-mu*v*(v.T.dot(b_k))

        norm = np.linalg.norm(b_k1)

        #normalize
        b_k = b_k1 / norm

    return b_k

def hotelling(A, v, mu, n_vectors, n_iterations):
    """Function to compute the eigenvectors that
    correspond to the second, third, ... , n_vectorsth
    largest eigenvalues along
    with their corresponding eigenvector. The largest eigenvalue
    and corresponding eigenvector need to have been computed first.
    Returns the eigenvectors as rows of an array and the eigenvalues
    as a list in descending order"""

    n = A.shape[1]
    #array for storing the eigenvectors as its rows
    V_T = np.array([v])

    #list for storing the eigenvalues
    mu_s = np.array(mu)

    #helper array for computations with mu*v as columns
    V_mu = np.reshape(mu*v, (-1,1))

    

    #each eigenvector and eigenvalue computed iteratively
    #using the previously computed ones
    for i in np.arange(n_vectors):
        
        #initial guess
        w0 = np.random.rand(n)
        norm = np.linalg.norm(w0)
        #normalize
        w0 = w0/norm
        
    
        #power iteration
        for j in np.arange(n_iterations):
            #Hotelling's deflation
            wj = A.dot(w0)-V_mu.dot(V_T.dot(w0))

            norm = np.linalg.norm(wj)

            #normalize
            w0 = wj / norm
        
        #calculate corresponding eigenvalue
        mu0 = rayleigh_quotient(A, w0)

        # store the eigenvector and eigenvalues 
        V_T = np.concatenate((V_T, np.array(w0)), axis = 0)
        mu_s.append(mu0)
        V_mu = np.column_stack((V_mu, mu0*w0))

    return V_T, mu_s 
   




#OBS! NEED TO CHECK IF THERE IS A DIFFERENCE IN SPEED WITH A.dot(v) vs. np.dot(A, v)



In [9]:
a = np.array([[1,2,3]])
b = np.array([[5,6,7]])
c = np.concatenate((a,b), axis = 0)

print(a.T.shape)
h = np.array([v1])
print(h.shape)

(3, 1)
(1, 41)


In [4]:
M = bio_1.values

v1 = power_iteration(M, 50)
mu1 = rayleigh_quotient(M,v1)
v2 = hotelling2(M,v1,mu1,50)
mu2 = rayleigh_quotient(M,v2)

V_T, mu_s = hotelling(M,v1,mu1,1,50)



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [13]:

WT = np.array([v1])
print(WT.shape)
W_mu = np.reshape(mu1*v1, (-1,1))
print(W_mu.shape)



(1, 41)
(41, 1)


In [10]:
w = hotelling(M,v1,mu1,1,50)
print(w.shape)

(41,)


In [31]:
b_k = np.random.rand(M.shape[1])
print(b_k.dtype == np.float64)

True


In [73]:

V = np.zeros(shape = (len(v1), 4))
V[:,0] = v1
print(V)

[[4.39649094e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.11128591e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.75410807e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.13533763e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.47374467e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.24459780e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.00212362e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.91260301e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.22695063e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.19081847e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.90017482e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.25001728e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.15144667e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.62641174e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.41288546e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.86376098e-04 0.00000000e+00 0.00000000e+00 0.000000

In [ ]:
def proj(v,w):
    """Compute the projection of v onto span(w).
    Parameters: v : list or 1-d numpy array
                w : list or 1-d numpy array"""
    p = v.dot(w)/(w.dot(w))*w
    return p

def perp(v, ws):
    """Compute the kohtisuora komponentti of v against subspace span(ws)
    Parameters: v : list or 1-d numpy array
                ws : a list of lists or 1-d numpy arrays. These have to
                be orthogonal"""
    projections = [proj(v,w) for w in ws]
    return v-np.sum(projections, axis=0)


Below the "real" eigenvalues and eigenvectors
----------------------------------------------

In [65]:


#compute eigenvalues and eigenvectors of M. Note: np.linalg.eigh gives eigenvalues in ascending order, 
# so the largest is last, and so is the corresponding eigenvector. 
# (NOTE: Don't need to worry abot how the order treats negative values as all eigenvalues of a positive semidefinite matrix are nonnegative)

evals, evecs = np.linalg.eigh(M) 
#largest eigenvalue and corresponding evec
eval1, evec1 = evals[-1], evecs[-1]
#second largest
eval2, evec2 = evals[-2], evecs[-2]


print(mu1)
print(eval1)
print(mu2)
print(eval2)


1.0168516885375931
1.0168519007503496
0.8866088779660348
0.8871946000095507
[ 2.22703320e+02 -7.52016596e+02 -5.73797612e-02  2.27502981e+01
  3.54008536e+01 -3.16010520e+01 -1.04212181e+02  8.91003912e+03
 -1.47986714e+03 -4.62706312e+00 -2.14176242e+01  2.64076308e+01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


In [70]:

print(v1.dot(v2))


-0.00011744376785258193


In [18]:

# check if the real and approximated eigenvector linearly dependent
A = np.stack((v_2, evec2), axis=1)
np.linalg.matrix_rank(A)



2

In [57]:
a_2 = rayleigh_quotient(M, v_2)
print(a_2)

11.694825610284667


In [7]:
for i in np.arange(len(evec1)):
    x = evec1[i]
    if x != 0.0:
        print(i,x)

11 1.0


In [51]:
for i in range(50,400,50):
    v_2 = hotelling2(M, v, a, i)
    a_2 = rayleigh_quotient(M, v_2)
    norm = residual_norm(M, v_2, a_2)
    print(f"Iteration {i}, residual norm: {norm}")
    

Iteration 50, residual norm: 0.002694597137400284
Iteration 100, residual norm: 0.00023027591081764133
Iteration 150, residual norm: 6.717010804712341e-05
Iteration 200, residual norm: 0.00014071462095937545
Iteration 250, residual norm: 0.00015732072384204554
Iteration 300, residual norm: 0.00018789944059598136
Iteration 350, residual norm: 1.0853296793970103e-06


In [49]:
#print(f"Real eigval: {eval1}")


for i in range(100,1000, 100):
    v = power_iteration(M, i)
    a = rayleigh_quotient(M, v)
    norm = residual_norm(M, v, a)
    print(f"Iteration: {i}, residual norm: {norm}")
    #print(f"Iteration: {i}, estimated eigval: {a}, residual norm: {norm}")
    
    

Iteration: 100, residual norm: 1.0367997085054587e-07
Iteration: 200, residual norm: 2.0375336947625995e-13
Iteration: 300, residual norm: 5.551169297415832e-17
Iteration: 400, residual norm: 5.551157474611588e-17
Iteration: 500, residual norm: 5.551157474611588e-17
Iteration: 600, residual norm: 5.551157474611588e-17
Iteration: 700, residual norm: 5.551157474611588e-17
Iteration: 800, residual norm: 5.551157474611588e-17
Iteration: 900, residual norm: 5.551157474611588e-17
